In [7]:
import requests
import base64
import os
import csv
import json
from rapidfuzz.distance import Levenshtein

In [8]:
# Configuration
DATASET_PATH = "/home/ribfli/Computer VIsion/ASS OCR WITH VLM/Optical-Chracter-Recognition-OCR--With-Visual-Language-Model-VLM-/data plate/Indonesian License Plate Recognition Dataset/images/test"
LABEL_CSV = "label.csv"
OUTPUT_CSV = "license_plate_ocr_results.csv"
SELECTED_MODEL_NAME = "gemma-3-4b-it-qat"  


In [9]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def calculate_cer(pred, truth):
    if not truth:
        return 1.0
    distance = Levenshtein.distance(pred, truth)
    cer = distance / len(truth)
    return round(cer, 4)


In [10]:
def load_ground_truth_from_csv(csv_path):
    gt_map = {}
    try:
        with open(csv_path, mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            for row in reader:
                if len(row) >= 2:
                    filename = os.path.basename(row[0]).strip()
                    label = row[1].strip().replace(" ", "").replace("-", "").upper()
                    gt_map[filename] = label
    except Exception as e:
        print(f" Failed to read CSV: {e}")
    return gt_map

In [11]:
def test_lmstudio_config():
    configs = [
        {"base_url": "http://localhost:1234", "endpoint": "/v1/chat/completions"},
        {"base_url": "http://127.0.0.1:1234", "endpoint": "/v1/chat/completions"},
        {"base_url": "http://localhost:8080", "endpoint": "/v1/chat/completions"},
    ]
    for config in configs:
        try:
            url = config["base_url"] + "/v1/models"
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                print(f"✅ Found working API at: {config['base_url']}")
                return config
        except Exception as e:
            print(f" Failed {config['base_url']}: {e}")
    return None


In [12]:
def main():
    print("🔍 Testing LMStudio configurations...")
    working_config = test_lmstudio_config()
    if not working_config:
        print(" LMStudio not running or API not reachable.")
        return

    base_url = working_config["base_url"]
    endpoint = working_config["endpoint"]
    model_name = SELECTED_MODEL_NAME

    print(f"🎯 Using model: {model_name}")
    print(f"📥 Loading ground truth from: {LABEL_CSV}")
    ground_truth_map = load_ground_truth_from_csv(LABEL_CSV)
    if not ground_truth_map:
        print("  No ground truth data found.")
        return

    if not os.path.exists(DATASET_PATH):
        print(f" Dataset path not found: {DATASET_PATH}")
        return

    print(f"\n🚀 Starting OCR processing...")
    with open(OUTPUT_CSV, mode='w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["image", "ground_truth", "prediction", "CER_score"])
        
        processed_count, error_count = 0, 0
        
        for root, _, files in os.walk(DATASET_PATH):
            for filename in files:
                if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                    if filename not in ground_truth_map:
                        print(f"⚠️ Ground truth not found for {filename}, skipping.")
                        continue

                    img_path = os.path.join(root, filename)
                    ground_truth = ground_truth_map[filename]
                    print(f"\n📸 Processing {filename}...")

                    try:
                        b64_image = encode_image(img_path)
                        payload = {
                            "model": model_name,
                            "messages": [
                                {
                                    "role": "user",
                                    "content": [
                                        {"type": "text", "text": "What is the license plate number shown in this image? Respond only with the plate number."},
                                        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{b64_image}"}}
                                    ]
                                }
                            ],
                            "temperature": 0.1,
                            "max_tokens": 50,
                            "stream": False
                        }
                        url = base_url + endpoint
                        headers = {
                            "Content-Type": "application/json",
                            "Accept": "application/json"
                        }
                        response = requests.post(url, json=payload, headers=headers, timeout=60)
                        if response.status_code != 200:
                            raise Exception(f"HTTP {response.status_code}: {response.text}")
                        
                        result = response.json()
                        if "choices" in result and len(result["choices"]) > 0:
                            prediction = result["choices"][0]["message"]["content"].strip()
                            prediction = prediction.replace(" ", "").replace("-", "").upper()
                            cer_score = calculate_cer(prediction, ground_truth)
                            print(f"✅ GT: {ground_truth} | Pred: {prediction} | CER: {cer_score}")
                            csv_writer.writerow([filename, ground_truth, prediction, cer_score])
                            processed_count += 1
                        else:
                            print(f" Unexpected response format")
                            csv_writer.writerow([filename, ground_truth, "FORMAT_ERROR", 1.0])
                            error_count += 1
                    except Exception as e:
                        print(f"Error on {filename}: {e}")
                        csv_writer.writerow([filename, ground_truth, "ERROR", 1.0])
                        error_count += 1

    print(f"\n Finished! Processed: {processed_count}, Errors: {error_count}")
    if processed_count > 0:
        try:
            with open(OUTPUT_CSV, 'r', encoding='utf-8') as csvfile:
                csv_reader = csv.DictReader(csvfile)
                cer_scores = [float(row['CER_score']) for row in csv_reader if float(row['CER_score']) < 1.0]
                if cer_scores:
                    avg_cer = sum(cer_scores) / len(cer_scores)
                    print(f"📈 Average CER Score: {avg_cer:.4f}")
        except Exception as e:
            print(f"⚠️ Failed to compute average CER: {e}")

if __name__ == "__main__":
    main()

🔍 Testing LMStudio configurations...
✅ Found working API at: http://localhost:1234
🎯 Using model: gemma-3-4b-it-qat
📥 Loading ground truth from: label.csv

🚀 Starting OCR processing...

📸 Processing test095_1.jpg...
✅ GT: B1201KAD | Pred: B1201KAD | CER: 0.0

📸 Processing test075_3.jpg...
✅ GT: 7066OF | Pred: 71066JF2425 | CER: 1.0

📸 Processing test023_1.jpg...
✅ GT: B1730JLO | Pred: B1730JLO | CER: 0.0

📸 Processing test078_1.jpg...
✅ GT: H1413DH | Pred: H1413DH0626 | CER: 0.5714

📸 Processing test034_2.jpg...
✅ GT: W1167V0 | Pred: W1167VA8 | CER: 0.2857

📸 Processing test031_2.jpg...
✅ GT: L1568Q0 | Pred: 156808 | CER: 0.4286

📸 Processing test095_2.jpg...
✅ GT: B2310JUK | Pred: B2310JUK | CER: 0.0

📸 Processing test033_1.jpg...
✅ GT: L4561VK | Pred: L4561VK0627 | CER: 0.5714

📸 Processing test075_1.jpg...
✅ GT: G2474XZ | Pred: G247412 | CER: 0.2857

📸 Processing test066_2.jpg...
✅ GT: H1790ZY | Pred: G428 | CER: 1.0

📸 Processing test001_2.jpg...
✅ GT: B2407UZO | Pred: B2407UZO | C